<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Natural-Language-Processing/blob/main/faker_news_classifier_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fake news classification

In [ ]:
# Importing the Pandas library as 'pd' to work with datasets
import pandas as pd


In [ ]:
# Load the dataset from a CSV file
data = pd.read_csv('/content/train.csv')

In [ ]:
# Show the first 5 rows of the dataset
data.head(5)

In [ ]:
# Display information about the dataset
data.info()

In [ ]:
# Display the dimensions of the dataset (rows, columns)
data.shape


In [ ]:
# Check for null values in the dataset
data.isnull().sum()

In [ ]:
# Remove rows with null values from the dataset
data = data.dropna()

In [ ]:
# Check for null values in the dataset
data.isnull().sum()

In [ ]:
# Display the dimensions of the dataset (rows, columns)
data.shape
